# OPL Ve DOcplex #
Eğitim ya da iş hayatında IBM CPLEX kullanan bir çok insan CPLEX'in kendi dili olan OPL kullanmıştır. Daha doğrusu OPL kullanarak CPLEX öğrenmeye başlamıştır. OPL'in artı ve eksi yönleri bulunuyor. <br><br>
Artıları, OPL'de model yazmak çok kolay. Eğer elinizde matematiksel bir gösterim bulunuyorsa bunu çok hızlı bir şekilde OPL'de yazabilirsiniz. SQL'de konuşur gibi yazıyorsunuz ya, OPL'de de aynı şekilde matetmatiksel gösterimin aynısını geçirebilirsiniz. OPL kullanımı ile ilgili de bir çok kaynak bulabilirsiniz. <br><br>
Eksiklere örnek verecek olursak,
<ol>
    <li>Veri kaynaklarına bağlantılarda sıkıntılar yaşayabilirsiniz</li>
    <li>CPLEX'in kendi Flow (akış) yönetimi biraz karışık. Bu kısmı öğrenmek için biraz çaba sarfetmek gerekiyor.</li>
    <li>OPL'de yazılan bir modeli ürünleştirmek için bir kaç takla atmak gerekebiliyor. (Oplrun gibi)</li>
    <li>Veri yapıları yeterli fakat veri tiplerinde bazı sıkıntılar yaşanabiliyor. <a href="https://community.ibm.com/community/user/datascience/communities/community-home/digestviewer/viewthread?MessageKey=551d41b7-8bf5-4f62-87c1-cacd889842ad">Örnek</a> </li>
</ol>

Eğitimin bu bölümünde, OPL'deki syntax ile DOcplex syntaxi karşılaştırmalı olarak anlatılacaktır. Bu anlatım için <a href="https://www.linkedin.com/in/alain-chabrier-5430656/?originalSubdomain=fr">Alain Chabrier'in</a> <a href="https://medium.com/@AlainChabrier/opl-vs-python-docplex-2d7b28814740">Medium'daki makalesi</a> örnek olarak kullanılacaktır. Bu notebook bir modeli baştan sona çözmek yerine, OPL ve DOcplex arasındaki kullanım farklarını anlatacaktır.
<br><br>

### Sigorta Fiyat Optimizasyonu ###
Yazılan modelin daha iyi anlaşılabilmesi için problemi de anlamak gerekiyor. Problemde, her bir müşterimiz için ciroyu maksimize edecek fiyatı teklifi vermeye çalışıyoruz. 

Modelde iki farklı veri bulunuyor.

<ul>
    <li><b>rangeAsSet.csv</b> Bu veri setinde müşteriler için verilen eski fiyatlar ile en düşük ve en yüksek fiyat limitleri bulunuyor.</li>
    <li><b>rawData.csv</b> Bu veri setinde, müşteriler için tüm fiyatlar için ihtimaller ve bu fiyatlara göre oluşacak cirolar bulunuyor.</li>
</ul>

Verileri, bu problemin çözümü için yazılmış modelleri incelemek için şu <a href="https://github.com/achabrier/assets/tree/master/InsurancePricing">linke</a> bakabilirsiniz. <br><br>

### Input Data - Modele verilerin alınması ###
OPL'de bir veritabanından ya da CSV'den veri aldığınız zaman __tuple__ yapısı kullanırsınız. Aşağıda OPL'de __rawData__ için hazırlanmış tuple bulunmaktadır.

<code><b>OPL</b> 
tuple TRawData {  
  int index;  
  key int customer;  
  key int priceIndex;  
  float price;  
  float probability;  
  float revenue;
};

{TRawData} rawData = ...; 
</code>



<b>DOcplex</b><br>
DOcplex bir python kütüphanesi olduğu için, veriyi python'ın özelliklerini kullanarak alacağız. Python'da veriyi işlemenin en kolay yolu nedir? Tabiki __pandas__. Pandas kullanarak csv'yi rahatlıkla dataframe atabiliyoruz. Aşağıda örneği bulunuyor.

In [39]:
import pandas as pd
rawData = pd.read_csv('rawData.csv')
rawData.head(2)

,index,customer,priceIndex,price,probability,revenue
0,0,0,0,0.0,0.977771,0.000000
1,1,0,1,1.0,0.973877,0.973877


Pandas'ın özelliklerini kullanarak, mevcut veri setimizi kullanarak farklı listeler oluşturabiliriz. Bu listeleri de kullanarak hem istemediğimiz alanları çıkarabilir ve yeni veri yapımız sayesinde oluşturacağımız KPI'lar için de daha verimli dataframeler oluşturabiliriz.

In [40]:
customers = rawData['customer'].unique().tolist()
priceIndices = rawData['priceIndex'].unique().tolist()
rawData = rawData.set_index(['customer', 'priceIndex'])

In [45]:
rawData.head(2)

index  price  probability   revenue
customer priceIndex                                     
0        0               0    0.0     0.977771  0.000000
         1               1    1.0     0.973877  0.973877

<br><br>
### Pre-proecessing - Veri Ön İşleme ###
Br optimizasyon modelindeki en büyük zorluklardan birisi, verinin optimizasyon için uygun olmamasıdır. Örneğin veri ambarı olmayan bir şirkette transaction (tekil kayıt) verilerinden optimizasyon yapmaya çalıştığınızı düşün. İlk yapmanız gereken verileri modelleme için gruplamanız (aggregate) gerekmektedir. Modelinizdeki indislerinize uygun master tablolalarınızı oluşturup, bu indislere göre kayıtlarınızı düzenlemeniz gerekmektedir. Genel olarak bu gibi işlemler veri kaynağındaki prosedürlerle yapmak daha doğrudur. Fakat küçük ya da daha az sıklıkla kullanılacak projelerde bu işlemler OPL ya da DOcplex (python) katmanında da yapılabilir. 

Bu eğitimin en başında bahsettiğim üzere, OPL'in esenk bir yapısı bulunuyor ve bu yapı sayesinde JS kodları bile kullanabiliyorsunuz. Tek satırda bir çok işlemi yapabiliyorsunuz. Aşağıda bir örnek ile giriş yapalım.

<code><b>OPL</b>
    {int} priceIndiceSubset =  { pi | pi in priceIndices : ord(priceIndices,pi) < card(priceIndices)-1};
</code>

PriceIndices kümesinde her bir müşteri için fiyat adımları tekrar etmektedir. Tüm tekil fiyat adımlarını almak için bu satırı yazıyoruz. __ord__ sırasını, __card__ küme içindeki sayıyı veriyor.

__DOcplex__ bu kodu aşağıdaki gibi yazıyoruz.


In [52]:
priceIndiceSubset =  [ pi for pi in priceIndices if priceIndices.index(pi)<len(priceIndices)-1 ]

<br><br>
### Model Creation - Model Oluşturma ###
OPL'de model oluşturmak için herhangi bir şey yapmanıza gerek yoktur. Zaten OPL yazıyor olmanız model yazdığınız anlamına gelir. Eğer kısıt problemi çözüyorsanız, OPL'in bunu anlaması için en başa <code>using CP</code> yazmanız gerkemektedir.

__DOcplex__'te model oluştururken ilgili kütüphaneyi import etmeniz gerekmektedir. Eğer Matematiksel Programlama çözecekseniz aşağıdaki yazmanız modele başlangıç yeterlidir.
    


In [46]:
from docplex.mp.model import Model 
mdl = Model(name='InsurancePricing')

DOcplex'in en güzel özelliklerinden biri, tek python dosyasınd abir çok model oluşturup bunları birleştirebilir, karşılaştırabilir, çıktılarını girdi olarak kullanabilirsiniz. OPL'de bunu yapmak için ayrı mod dosyaları oluşturmanız gerekmektedir. Bunları karşılaştırmak ya da birleştirmek için de ayrı bir flow oluşturmanız gerekecektir.

<br><br>
### Decision Variables - Karar Değişkenleri ###
OPL'de karar değişkeni oluşturmak için __dvar__ yazmanız ardından veri tipini belirttikten sonra indisleri yazmanız gerekir.
<code><b>OPL</b><br>
    dvar  float lambda1[c in customers][pi in priceIndices] in 0..1;
    dvar  float lambda2[c in customers][pi in priceIndices] in 0..1;
    dvar  boolean z[c in customers][pi in priceIndiceSubset];
</code>
   
<b>DOcplex</b>'de ise önceki derslerimizde anlattığımız gibi, daha önce oluşturduğumuz model (mdl) ekleme yaparak ilerliyoruz. binary, integer ya da continuous değişkenler ile eklemeler yapıyoruz. Modelimizde 2 sürekli bir de ikili değişkenimiz bulunuyor. İkili değişkenimiz hangi müşteri için hangi fiyatı kullanacağımızı bulmak için kullanacağız. 

In [ ]:
lambda1 = mdl.continuous_var_matrix(customers, priceIndices, lb=0, ub=1, name='lambda1')
lambda2 = mdl.continuous_var_matrix(customers, priceIndices, lb=0, ub=1, name='lambda2')
z = mdl.binary_var_matrix(customers, priceIndiceSubset, name='z')

<br><br>
### KPIs - Decission Expressions - Performans Metrikleri ###
__OPL__'de model sonucunun performansının daha iyi ölçülmesi, metriklerin hespalanması ve ama ç fonksiyonun daha kolay yazılması için __dexpr__ ifadesi bulunmaktadır. __dexpr__ sayesinde hesaplamalar daha kolay yapılmakta ve bu hesaplamalar modelin çıktısına ek olarak gösterilebilmektedir. Aşağıda örnek __dexpr__ yazılışları bulunmaktadır.

<code><b>OPL</b>
    dexpr float Revenue = revenue;
    dexpr float Volume = volume ;
    dexpr float AvgPriceIncrease = averagePriceIncrease;
</code>


__DOcplex__'de modele ekleme yapıyoruz. Öncelikle modelde hesap yapmak istediğimiz KPI ilgili agregate fonksiyonu ile ekledikten sonra __add_kpi__ yazarak bu KPI eklemesini yapıyoruz. Aşağıda örnek KPI eklemesini görebilirsiniz.


In [ ]:
averagePriceIncrease = mdl.sum(((priceApplied[c]-previousPrice[c])/previousPrice[c])/len(customers) for c in customers)
mdl.add_kpi(averagePriceIncrease, publish_name="KPI.AvgPriceIncrease")

<br><br>
### Objectives - Amaç Fonksiyonu ###
Amaç fonksiyonu yazmak hem OPL'de hem de DOcplex'de gayet kolaydır. İncelediğimiz bu modelde ağırlıklar kullanılmaktadır. Bu sebeple bu ağırlıklar kullanılarak, Ciro (revenue), Hacim (Volume), Ortalama Fiyat Artışı(Avg Price Increase) toplamı minimize edilmeye çalışılmaktadır. __Weights__ adlı tablodaki ağırlıklar değiştirilerek model sonucu gözlemlenmektedir. Bu sebeple öncelikle bu ağırlıkların kullanıldığı KPI'lar oluşturulmakta ve bunların toplamı minimize edilmektedir.

<code><b>OPL</b>
    dexpr float resRevenue = - revenueWeight * revenue;
    dexpr float resVolume = - volumeWeight * volume ;
    dexpr float resAvgPriceIncrease = avgIncWeight * averagePriceIncrease;minimize resRevenue + resVolume + resAvgPriceIncrease;
</code>

__DOcplex__'de OPL'de yazdığımızdan bir farkı bulunmamaktadır. Öncelikle değişkenlere atamalar yapıldıktan sonra, amaç fonksiyonu modele eklenmektedir.


In [ ]:
resRevenue = - revenueWeight * revenue;
resVolume = - volumeWeight * volume ;
resAvgPriceIncrease = avgIncWeight * averagePriceIncrease;mdl.minimize(resRevenue + resVolume + resAvgPriceIncrease)

<br><br>
### Constraints - Kısıtlar ###
__OPL__'de kısıt yazarken büyük kolaylık sağlayan __forall__ ifadesi bulunmaktadır. Kısıdı yazdığımız duruma göre __sum__ ile birlikte kullanabilmekteyiz. Bu sayede __her bir müşterinin__ ya da __her bir müşterinin alışverilerinin toplamı__ gibi ifadeleri kolaylıkla yazabilmekteyiz. Aşağıda örnek bir yazım bulunmaktadır. 

<code><b>OPL</b>
forall( c in customers, pi in priceIndiceSubset) {            
  ctConvexityCondition:        
    lambda1[c][pi] + lambda2[c][pi] - z[c][pi] == 0;  
}
</code>


__DOcplex__'de ise Python'ın döngü araçlarından biri olan __for__ kullanılmaktadır. __OPL__'de __forall__ parantezinde tüm indisler için tek seferde döndürebilmekteydik. Fakat Python'dan her bir indis için iç içe for yazmamız gerekmektedir. Eğer toplamlar için bir kısıt yazmamız gerekiyorsa __mdl.sum__ ifadesi ile kontrollerimizi yapmamız gerekiyor. Yazdığımız kısıtları modele eklemek için de __mdl.add_constraint()__ ifadesini kullanıyoruz.

In [ ]:
for c in customers:    
  for pi in priceIndiceSubset:        
    mdl.add_constraint(lambda1[c, pi] + lambda2[c, pi] - z[c, pi] == 0, 'ctConvexityCondition')

<br><br>
### Parameters - Parametreler ###
CPLEX'de modelin çalışma süresi, gap limiti, vb. bir çok parametre hem OPL'de hem de DOcplex'de kullanımı çok kolaydır. CPLEX'de kullanılan parametreler için IBM çok detaylı ve düzenli bir dokuman yayınlamıştır. CPLEX kullanan herkesin bu dokumanı incelemesini tavsiye ederim. Özellikle uzun süren ve bitmeyen optmizasyon modellerinizi buradaki parametreleri inceleyerek ve kullanarak çok hızlı bir şekilde çözüme ulaştırabilirsiniz. Buraya <a href="https://www.ibm.com/docs/en/SSSA5P_12.8.0/ilog.odms.studio.help/pdf/paramcplex.pdf">linki</a> bırakıyorum. Umarım faydalı olur. 

<code><b>OPL</b>
execute PARAMS {
  cplex.tilim = 100;
}
</code>


__DOcplex__'de parametreler modele __mdl.parameters__ ile eklenir.

mdl.parameters.mip.tolerances.mipgap = 0.2

<br><br>
### Solve - Çözüm ###
__OPL__'de çözmek için herhangi ifade yazmanıza gerek yoktur. IDE üzerinden çalıştır düğmesine bastığınızda model çalışır ve sonuçlar ekrana ya da istediğiniz dosyaya yazarak son bulur. 

__DOcplex__'de ise __mdl.solve()__ ifadesini kullanmaktayız. Bu ifade ile model çözmeye başlayacaktır.

In [ ]:
ok = mdl.solve()

<br><br>
### Sonuç ###
Bu eğitimimizde __OPL__ ile __DOcplex__ arasındaki farkları Alain Chabrier'in makalesinden faydalanarak, biraz da kendimizden birşeyler ekleyerek anlatmaya çalıştık. Umarım faydalı olmuştur. Bir sonraki eğitimde görüşmek üzere.<br>
__Sabri Suyunu__
